In [36]:
# 多头注意力机制就是对原来的Q分割为Q1,Q2等，KV同理
import math
import torch
from torch import nn
from d2l import torch as d2l

In [37]:
# 对输入进行多头分割
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 例如：输入（4，16，64）——输出（4，16，4，16）
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 变为（4，4，16，16）
    X = X.permute(0, 2, 1, 3)

    # 输出合并前两个维度（16，16，16）
    return X.reshape(-1, X.shape[2], X.shape[3])

# 上面操作的逆转
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [38]:
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，
        # num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        # 没搞懂干嘛的
        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        # output的形状:(batch_size*num_heads，查询的个数，
        # num_hiddens/num_heads)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [57]:
# 测试上面的函数
num_hiddens, num_heads = 128, 4
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=128, out_features=128, bias=False)
  (W_k): Linear(in_features=128, out_features=128, bias=False)
  (W_v): Linear(in_features=128, out_features=128, bias=False)
  (W_o): Linear(in_features=128, out_features=128, bias=False)
)

In [60]:
# 输入序列的张端不一致并不重要，不影响计算
# 这里X（2，4，100） ， Y（2，6，100）
batch_size, num_queries = 2, 16
num_kvpairs, valid_lens = 4, torch.tensor([3, 2])
# X是当前的query
X = torch.ones((batch_size, num_queries, num_hiddens))
# Y是别人key,value
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape
# 为什么改变batch_size就报错？？

torch.Size([2, 16, 128])